### The data has been generated in the multi_mackend_madel_data folder, you can choose not to run this code file

In [1]:
from qiskit_ibm_provider import IBMProvider
from qiskit import Aer
from qiskit import QuantumCircuit, transpile, ClassicalRegister
from qiskit.circuit import Operation
from qiskit_aer.noise import NoiseModel
from qiskit_aer import AerSimulator
from qiskit.providers.models import BackendProperties
from qiskit.tools.visualization import plot_histogram
from qiskit.visualization import dag_drawer
from qiskit.converters import *
from qiskit.dagcircuit import dagnode
import os
import pickle
import numpy as np
import pandas as pd
import json
from tqdm.notebook import tqdm
from scipy import stats
from collections import defaultdict
from qiskit.providers.fake_provider import FakeMontreal, FakeLima,FakeGuadalupe,FakeSherbrooke,FakePrague,FakeCairo
import glob,copy
# with open("API_KEY.txt","r") as file:
#     key = file.read()
# provider = IBMProvider(token=key)
# backends = provider.backends(filters=lambda x: x if "simulator" not in x.name else None)
# backends = sorted(backends,key=lambda x: x.name)
sim_ideal = AerSimulator(seed_simulator=42)
# backends
backends = [FakeGuadalupe()]

C:\Users\30806\AppData\Local\Temp\ipykernel_26632\1533037012.py:1: DeprecationWarning: The package qiskit_ibm_provider is being deprecated. Please see https://docs.quantum.ibm.com/api/migration-guides/qiskit-runtime to get instructions on how to migrate to qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  from qiskit_ibm_provider import IBMProvider


In [2]:
# 设置理想和含噪声的后端参数
from qiskit.providers.fake_provider import FakeGuadalupe,FakeMontreal,FakeCairo,FakeMumbai,FakeSydney,FakeToronto
backend_list = []
# properties_dict = {}
# run_config_ideal_list = []
# run_config_noisy_list = []
backend = FakeGuadalupe()
backend_list.append(backend)
backend = FakeMontreal()
backend_list.append(backend)
backend = FakeCairo()
backend_list.append(backend)
backend = FakeMumbai()
backend_list.append(backend)
backend = FakeSydney()
backend_list.append(backend)
backend = FakeToronto()
backend_list.append(backend)
# for backend in backend_list:
#     # properties = get_backend_properties_v1(backend)
#     backend_name = backend.name()
#     # properties_dict[backend_name] = properties
#     # backend_ideal = QasmSimulator() # Noiseless
#     backend_noisy = AerSimulator.from_backend(backend)
#     shots = 100000
#     # run_config_ideal = {'shots': shots, 'backend': backend_ideal, 'name': 'ideal'}
#     # run_config_ideal_list.append(run_config_ideal)
#     run_config_noisy = {'shots': shots, 'backend': backend_noisy, 'name': 'noisy'}
#     run_config_noisy_list.append(run_config_noisy)

In [3]:
def update_noisy_dict(dict_1,dict_2,dict_3):
    dicts = [dict_1,dict_2,dict_3]
    keys = set().union(*dicts)
    for d in dicts:
        for key in keys:
            if key not in d:
                d[key] = 1
    dict_1 = dicts[0]
    dict_2 = dicts[1]
    dict_3 = dicts[2]
    return dict_1,dict_2,dict_3


# 加载训练和测试数据文件
def load_circuits(data_files, f_ext='.pk'):
    backend_name_list = []
    circuits = []
    ideal_exp_vals = []
    noisy_exp_vals_1 = []
    noisy_exp_vals_2 = []
    noisy_exp_vals_3 = []
    for data_file in data_files:
        if f_ext == '.json':
            for entry in json.load(open(data_file, 'r')):
                # 获取当前线路对应的后端，这个后端名称可以从数据文件的名字获得
                data_backend_name = os.path.basename(data_file).split('.')[0].split('_')[0] + '_' + os.path.basename(data_file).split('.')[0].split('_')[1]
                backend_name_list.append(data_backend_name)
                circuits.append(QuantumCircuit.from_qasm_str(entry['circuit']))
                ideal_exp_vals.append(entry['ideal_exp_value'])
                noisy_exp_vals_1.append(entry['noisy_exp_values_1'])
                noisy_exp_vals_2.append(entry['noisy_exp_values_2'])
                noisy_exp_vals_3.append(entry['noisy_exp_values_3'])
        elif f_ext == '.pk':
            for entry in pickle.load(open(data_file, 'rb')):
                # 获取当前线路对应的后端，这个后端名称可以从数据文件的名字获得
                data_backend_name = os.path.basename(data_file).split('.')[0].split('_')[0] + '_' + os.path.basename(data_file).split('.')[0].split('_')[1]
                backend_name_list.append(data_backend_name)
                circuits.append(entry['circuit'])
                ideal_exp_vals.append(entry['ideal_exp_value'])
                noisy_exp_vals_1.append(entry['noisy_exp_values_1'])
                noisy_exp_vals_2.append(entry['noisy_exp_values_2'])
                noisy_exp_vals_3.append(entry['noisy_exp_values_3'])
    return circuits, ideal_exp_vals, noisy_exp_vals_1,noisy_exp_vals_2,noisy_exp_vals_3,backend_name_list

def read_backend_data(filename):
    file = open(filename,"rb")
    data = pickle.load(file)
    file.close()
    return data

def read_and_transpile_circuit(filename,hardware,inputvalue=None):
    if inputvalue!=None:
        if inputvalue=="-1":
            circ = QuantumCircuit.from_qasm_file(filename)
            inputvalue = "1"*len(circ.qubits)
            
        circ = QuantumCircuit.from_qasm_file(filename)

        if len(inputvalue)<len(circ.qubits):
            padding = "0"*(len(circ.qubits)-len(inputvalue))
            inputvalue = padding+inputvalue
            
        full_circ = QuantumCircuit()
        for reg in circ.qregs:
            full_circ.add_register(reg)
        for reg in circ.cregs:
            full_circ.add_register(reg)
        for i,inp in enumerate(inputvalue):
            if inp=="1":
                full_circ.x(i)

        full_circ = full_circ.compose(circ)
        circ_trans = transpile(full_circ, hardware,optimization_level=0)
    else:
        circ = QuantumCircuit.from_qasm_file(filename)
        circ_trans = transpile(circ, hardware,optimization_level=0)
    return circ, circ_trans

def convert_circuit_to_graph(circ_tanspiled):
    graph = circuit_to_dag(circ_tanspiled)
    return graph


def get_inverse_circuit(circ_trans,graph):
    measurement_qubits = [(circ_trans.find_bit(node.qargs[0]).index,circ_trans.find_bit(node.cargs[0]).index) for node in graph.op_nodes() if node.op.name=="measure"]
    circ_trans_reverse = circ_trans.remove_final_measurements(inplace=False)
    inv = circ_trans_reverse.inverse()
    circ_trans_reverse = circ_trans_reverse.compose(inv)
    circ_trans_reverse.add_register(ClassicalRegister(circ_trans.num_clbits))
    circ_trans_reverse.barrier(circ_trans_reverse.qubits)
    for mindex in measurement_qubits:
        circ_trans_reverse.measure(mindex[0],mindex[1])
    
    return circ_trans_reverse

def get_circuit_measurements(circ_trans,graph):
    measurement_qubits = [(circ_trans.find_bit(node.qargs[0]).index,circ_trans.find_bit(node.cargs[0]).index) for node in graph.op_nodes() if node.op.name=="measure"]
    return measurement_qubits
    
def execute_with_repitions(circuit, simulator ,repitions=50):
    # circ_list = [circuit for x in range(repitions)]
    states = []
    for _ in range(repitions):
        state = simulator.run(circuit,shots=100000).result().get_counts()
        states.append(state) 
    # states = simulator.run(circ_list,shots=100000).result().get_counts()
    shots = sum(states[0].values())
    return [dict([(k,v/shots) for k,v in x.items()]) for x in states]


# def split_circuit(circuit, start, end, graph):
#     nq = len(circuit.qubits)
#     qc2 = QuantumCircuit(nq)
#     for x in circuit[start:end]:
#         qc2.append(x[0], x[1])
    
#     inv = qc2.inverse()
#     qc2 = qc2.compose(inv)
#     measurement_qubits = get_circuit_measurements(circuit,graph)
#     qc2.add_register(ClassicalRegister(circuit.num_clbits))
#     for mindex in measurement_qubits:
#         qc2.measure(mindex[0],mindex[1])
#     return qc2

def split_circuit(circuit, start, end, graph):
    qregs = circuit.qregs  # 获取原始线路中使用的量子寄存器
    cregs = circuit.cregs  # 获取原始线路中使用的经典寄存器
    qc2 = QuantumCircuit(*qregs, *cregs)  # 创建一个新的量子线路，使用与原线路相同的寄存器
    for x in circuit[start:end]:
        qc2.append(x[0], x[1])
    inv = qc2.inverse()
    qc2 = qc2.compose(inv)
    measurement_qubits = get_circuit_measurements(circuit,graph)
    for mindex in measurement_qubits:
        qc2.measure(mindex[0],mindex[1])
    return qc2

def split_circuit_to_percentile(circ_trans,graph):
    try:
        p1 = int(np.percentile([x for x in range(len(circ_trans))],25))
        p2 = int(np.percentile([x for x in range(len(circ_trans))],50))
        p3 = int(np.percentile([x for x in range(len(circ_trans))],75))
        qc1 = split_circuit(circ_trans,0,p1,graph)
        qc2 = split_circuit(circ_trans,p1,p2,graph)
        qc3 = split_circuit(circ_trans,p2,p3-1,graph)
        return qc1,qc2,qc3
    except:
        return False,True,False
    


def get_gate_info(circ, dag):
    
    features = {"Num_1Q_Gates":0,
                "Num_2Q_Gates":0}
    
    for node in dag.nodes():
            try:
                if node.qargs:
                    if node.name=="barrier" or node.name=="measure":
                        continue
                    if len(node.qargs)==1:
                        features["Num_1Q_Gates"]+=1
                    else:
                        features["Num_2Q_Gates"]+=1
            except:
                pass
    return features
    
def HellingerDistance(p, q):
    n = len(p)
    sum_ = 0.0
    for i in range(n):
        sum_ += (np.sqrt(p[i]) - np.sqrt(q[i]))**2
    result = (1.0 / np.sqrt(2.0)) * np.sqrt(sum_)
    return result
    
def generate_state_features(state,noise_state_runs):
    state_features = {}
    observed_prob_25 = np.round(np.percentile([x[state] for x in noise_state_runs if state in list(x.keys())],25),5)
    observed_prob_50 = np.round(np.percentile([x[state] for x in noise_state_runs if state in list(x.keys())],50),5)
    observed_prob_75 = np.round(np.percentile([x[state] for x in noise_state_runs if state in list(x.keys())],75),5)
    odds_ratio = [x[state]/(1-x[state]) for x in noise_state_runs if state in list(x.keys())]
    Avg_odds_ratio = np.round(max(odds_ratio)/min(odds_ratio),5)
    Avg_observed_prob = np.round(np.max([x[state] for x in noise_state_runs if state in list(x.keys())]),5)
    state_weight = len([x for x in state if x=="1"])
    state_features["Avg_odds_ratio"] = np.round(Avg_odds_ratio,2)#
    state_features["state_weight"] = state_weight#
    state_features["observed_prob_25"] = np.round(observed_prob_25*100,2)#
    state_features["observed_prob_50"] = np.round(observed_prob_50*100,2)#
    state_features["observed_prob_75"] = np.round(observed_prob_75*100,2)#
    return state_features

In [200]:
# 筛选出没法分割的线路并替换，不用再执行
paths = './multi_backend_model_data/test_1'
data_paths = glob.glob(f"{paths}/**.qasm")
for qasm in data_paths:
    circuit = QuantumCircuit.from_qasm_file(qasm)
    print(qasm)
    qubits = qasm.split('_')[-3]
    backend_name = qasm.split('_')[-5].split('\\')[-1] + '_' +qasm.split('_')[-4]
    print(qubits)
    print(backend_name)
    try:
        p1 = int(np.percentile([x for x in range(len(circuit))],25))
        p2 = int(np.percentile([x for x in range(len(circuit))],50))
        p3 = int(np.percentile([x for x in range(len(circuit))],75))
        print(p1)
        graph = convert_circuit_to_graph(circuit)
        qc1 = split_circuit(circuit,0,p1,graph)
        qc2 = split_circuit(circuit,p1,p2,graph)
        qc3 = split_circuit(circuit,p2,p3,graph)
    except:
        for backend_i in backend_list:
            if (backend_i.name() == backend_name):
                backend = backend_i
        circuit = transpile(
                    random_circuit(int(qubits), 5, measure=True),
                    backend,
                    optimization_level=3,
                )
        while(len(circuit)<10):
            circuit = transpile(
                        random_circuit(int(qubits), 5, measure=True),
                        backend,
                        optimization_level=3,
                    )
        qasm_circuit = circuit.qasm().strip()
        with open(qasm,'w') as file:
                file.write(qasm_circuit)

./multi_backend_model_data/test_1\fake_guadalupe_10_qubits_0.qasm
10
fake_guadalupe
66
./multi_backend_model_data/test_1\fake_guadalupe_10_qubits_1.qasm
10
fake_guadalupe
26
./multi_backend_model_data/test_1\fake_guadalupe_10_qubits_10.qasm
10
fake_guadalupe
64
./multi_backend_model_data/test_1\fake_guadalupe_10_qubits_11.qasm
10
fake_guadalupe
13
./multi_backend_model_data/test_1\fake_guadalupe_10_qubits_12.qasm
10
fake_guadalupe
26
./multi_backend_model_data/test_1\fake_guadalupe_10_qubits_13.qasm
10
fake_guadalupe
44
./multi_backend_model_data/test_1\fake_guadalupe_10_qubits_14.qasm
10
fake_guadalupe
53
./multi_backend_model_data/test_1\fake_guadalupe_10_qubits_15.qasm
10
fake_guadalupe
83
./multi_backend_model_data/test_1\fake_guadalupe_10_qubits_16.qasm
10
fake_guadalupe
12
./multi_backend_model_data/test_1\fake_guadalupe_10_qubits_17.qasm
10
fake_guadalupe
39
./multi_backend_model_data/test_1\fake_guadalupe_10_qubits_18.qasm
10
fake_guadalupe
22
./multi_backend_model_data/test_1\

In [4]:
def generate_training_data():
    for backend in tqdm(backend_list,position=0):
#         datafiles = os.listdir(f"./raw_data/{backend.name}")
#         np.random.shuffle(datafiles)
        # dataframe = pd.DataFrame()
#        for datafile in datafiles:
#             data = read_backend_data(os.path.join(f"./raw_data/{backend.name}",datafile))
        backend_name = backend.name()
        save_path = f"./multi_backend_model_data/Q_learn_train/training_{backend.name()}.csv"
        noise_model = NoiseModel.from_backend(backend)
        sim_noise = AerSimulator(noise_model=noise_model)
        
        data_path = './multi_backend_model_data/train_data'
        data_paths = glob.glob(f"{data_path}/**.pk")
        train_paths = []
        # 只获取当前后端对应的数据文件
        for path in data_paths:
            backend_name_i = os.path.basename(path).split('_')[0] + '_' + os.path.basename(path).split('_')[1]
            if (backend_name_i == backend_name):
                train_paths.append(path)
        # 此时只加载对应后端数数据文件
        circuits, ideal_exp_vals, noisy_exp_vals_1,noisy_exp_vals_2,noisy_exp_vals_3,backend_name_list = load_circuits(train_paths)
        # 将数据按照相同的次序打乱
        circuits = np.array(circuits)
        ideal_exp_vals = np.array(ideal_exp_vals)
        noisy_exp_vals_1 = np.array(noisy_exp_vals_1)
        noisy_exp_vals_2 = np.array(noisy_exp_vals_2)
        noisy_exp_vals_3 = np.array(noisy_exp_vals_3)
        # backend_name_list = np.array(backend_name_list)
        rng = np.random.default_rng()  # 创建随机数生成器
        indices = rng.permutation(len(circuits))
        circuits = circuits[indices]
        ideal_exp_vals = ideal_exp_vals[indices]
        noisy_exp_vals_1 = noisy_exp_vals_1[indices]
        noisy_exp_vals_2 = noisy_exp_vals_2[indices]
        noisy_exp_vals_3 = noisy_exp_vals_3[indices]
        # backend_name_list = backend_name_list[indices]
        
        for cir_index,circuit in enumerate(circuits):
            #print(circuitfile)
            circ = circuit
            circ_trans = circuit
            graph = convert_circuit_to_graph(circ_trans)
            ideal = ideal_exp_vals[cir_index]
            # 将三个字典给处理成key相同的情况
            noisy_exp_vals_1[cir_index],noisy_exp_vals_2[cir_index],noisy_exp_vals_3[cir_index] = update_noisy_dict(noisy_exp_vals_1[cir_index],noisy_exp_vals_2[cir_index],noisy_exp_vals_3[cir_index])
            # 取起其中一个
            noisy_exp_vals_1[cir_index] = {k: v/100000 for k, v in noisy_exp_vals_1[cir_index].items()}
            noisy_exp_vals_2[cir_index] = {k: v/100000 for k, v in noisy_exp_vals_2[cir_index].items()}
            noisy_exp_vals_3[cir_index] = {k: v/100000 for k, v in noisy_exp_vals_3[cir_index].items()}
            noisy = noisy_exp_vals_1[cir_index]
            graph = convert_circuit_to_graph(circ_tanspiled=circ_trans)
            graph2 = convert_circuit_to_graph(circ)
            qc1,qc2,qc3 = split_circuit_to_percentile(circ,graph2)
            original=True
            if qc1==None:
                qc1,qc2,qc3 = split_circuit_to_percentile(circ_trans,graph)
                original=False
            if original:
                qc1 = transpile(qc1,backend)
                qc2 = transpile(qc2,backend)
                qc3 = transpile(qc3,backend)
            
            qc1_state_runs = execute_with_repitions(qc1,simulator=sim_noise,repitions=2)[0]
            qc2_state_runs = execute_with_repitions(qc2,simulator=sim_noise,repitions=2)[0]
            qc3_state_runs = execute_with_repitions(qc3,simulator=sim_noise,repitions=2)[0]
            # generate all features
            
            state_features = get_gate_info(circ,graph)
            #############################################################################
            try:
                P = []
                Q = []
                for inv_keys in qc1_state_runs.keys():
                    if sum([int(x) for x in inv_keys if x!=' '])==0:
                        P.append(1)
                    else:
                        P.append(0)
                    Q.append(qc1_state_runs[inv_keys])
                
                Avg_inverted_error_25 = HellingerDistance(P,Q)
                
                P = []
                Q = []
                for inv_keys in qc2_state_runs.keys():
                    if sum([int(x) for x in inv_keys if x!=' '])==0:
                        P.append(1)
                    else:
                        P.append(0)
                    Q.append(qc2_state_runs[inv_keys])
                
                Avg_inverted_error_50 = HellingerDistance(P,Q)
                
                P = []
                Q = []
                for inv_keys in qc3_state_runs.keys():
                    if sum([int(x) for x in inv_keys if x!=' '])==0:
                        P.append(1)
                    else:
                        P.append(0)
                    Q.append(qc3_state_runs[inv_keys])
                
                Avg_inverted_error_75 = HellingerDistance(P,Q)
                
            except Exception as e:
                # print(inverted_state_runs)
                raise(e)
            circuit_depth = circ_trans.depth()
            circuit_width = circ_trans.width()
            ##############################################################################
            state_features["circuit_depth"] = circuit_depth
            state_features["circuit_width"] = circuit_width
            # state_features["Avg_inverted_error"] = np.round(Avg_inverted_error*100,2)
            
            #state_features["Avg_observed_prob"] = Avg_observed_prob
            state_features["Avg_inverted_error_25"] = np.round(Avg_inverted_error_25*100,2)
            state_features["Avg_inverted_error_50"] = np.round(Avg_inverted_error_50*100,2)
            state_features["Avg_inverted_error_75"] = np.round(Avg_inverted_error_75*100,2)
            # 取三次结果中的2次结果组成这个数组
            noise_state_runs = [noisy_exp_vals_2[cir_index],noisy_exp_vals_3[cir_index]]
            for state in noisy.keys():
                #try:
                # print(noise_state_runs)
                # print(ideal)
                state_features_state = generate_state_features(state,noise_state_runs)
                state_features.update(state_features_state)
                if state in ideal.keys():
                    state_features["target"] = np.round((ideal[state]/100000)*100,2)
                else:
                    state_features["target"] = 0.0
                
                
                dataframe = pd.DataFrame(state_features,index=[0])
                if  not  os.path.exists(save_path):
                    dataframe.to_csv(save_path,  index=False,  mode='w',  header=True)
                else:
                    dataframe.to_csv(save_path,  index=False,  mode='a',  header=False)
        del sim_noise




In [5]:
def generate_test_data():
    for backend in tqdm(backend_list,position=0):
        backend_name = backend.name()
        noise_model = NoiseModel.from_backend(backend)
        sim_noise = AerSimulator(noise_model=noise_model)
        # The test data comes from random lines or MQT test sets
        data_path = './multi_backend_model_data/MQT_test'
        # data_path = './multi_backend_model_data/test_data'
        data_paths = glob.glob(f"{data_path}/**.pk")
        test_paths = []
        
        for path in data_paths:
            backend_name_i = os.path.basename(path).split('_')[0] + '_' + os.path.basename(path).split('_')[1]
            if (backend_name_i == backend_name):
                test_paths.append(path)
        # 此时只加载对应后端数数据文件
        circuits, ideal_exp_vals, noisy_exp_vals_1,noisy_exp_vals_2,noisy_exp_vals_3,backend_name_list = load_circuits(test_paths)
        # 将数据按照相同的次序打乱
        circuits = np.array(circuits)
        ideal_exp_vals = np.array(ideal_exp_vals)
        noisy_exp_vals_1 = np.array(noisy_exp_vals_1)
        noisy_exp_vals_2 = np.array(noisy_exp_vals_2)
        noisy_exp_vals_3 = np.array(noisy_exp_vals_3)
        rng = np.random.default_rng()  # 创建随机数生成器
        indices = rng.permutation(len(circuits))
        circuits = circuits[indices]
        ideal_exp_vals = ideal_exp_vals[indices]
        noisy_exp_vals_1 = noisy_exp_vals_1[indices]
        noisy_exp_vals_2 = noisy_exp_vals_2[indices]
        noisy_exp_vals_3 = noisy_exp_vals_3[indices]

        for cir_index,circuit in enumerate(circuits):
            circ = circuit
            circ_trans = circuit
            graph = convert_circuit_to_graph(circ_trans)
            ideal = ideal_exp_vals[cir_index]
            qubits = len(next(iter(ideal)))
            noisy_exp_vals_1[cir_index],noisy_exp_vals_2[cir_index],noisy_exp_vals_3[cir_index] = update_noisy_dict(noisy_exp_vals_1[cir_index],noisy_exp_vals_2[cir_index],noisy_exp_vals_3[cir_index])
            noisy_exp_vals_1[cir_index] = {k: v/100000 for k, v in noisy_exp_vals_1[cir_index].items()}
            noisy_exp_vals_2[cir_index] = {k: v/100000 for k, v in noisy_exp_vals_2[cir_index].items()}
            noisy_exp_vals_3[cir_index] = {k: v/100000 for k, v in noisy_exp_vals_3[cir_index].items()}
            noisy = noisy_exp_vals_1[cir_index]
            state_len = len(noisy.values())
            
            #####################################################
            # conver transpiled circuit to DAG
            graph = convert_circuit_to_graph(circ_tanspiled=circ_trans)
            graph2 = convert_circuit_to_graph(circ)
            # divide transpiled circuit to 3 percentiles inverted circuit
            qc1,qc2,qc3 = split_circuit_to_percentile(circ,graph2)
            if (not qc1 and qc2 and not qc3):
                continue 
            original=True
            noise_state_runs = execute_with_repitions(circ_trans,simulator=sim_noise,repitions=2)
            
            # circ_trans_inverted = transpile(circ_trans_inverted,backend)
            if original:
                qc1 = transpile(qc1,backend)
                qc2 = transpile(qc2,backend)
                qc3 = transpile(qc3,backend)
            
            # inverted_state_runs = execute_with_repitions(circ_trans_inverted,simulator=sim_noise,repitions=2)[0]
            qc1_state_runs = execute_with_repitions(qc1,simulator=sim_noise,repitions=2)[0]
            qc2_state_runs = execute_with_repitions(qc2,simulator=sim_noise,repitions=2)[0]
            qc3_state_runs = execute_with_repitions(qc3,simulator=sim_noise,repitions=2)[0]
            # generate all features
            

            
            
            get_gate = get_gate_info(circ,graph)
            Num_1Q_Gates = get_gate['Num_1Q_Gates']
            Num_2Q_Gates = get_gate['Num_2Q_Gates']
            state_features = {}
            state_features['Num_1Q_Gates'] = []
            state_features['Num_2Q_Gates'] = []
            state_features['Num_1Q_Gates'][0:state_len] = [Num_1Q_Gates]*state_len
            state_features['Num_2Q_Gates'][0:state_len] = [Num_2Q_Gates]*state_len
            
            #############################################################################
            try:
                
                P = []
                Q = []
                for inv_keys in qc1_state_runs.keys():
                    if sum([int(x) for x in inv_keys if x!=' '])==0:
                        P.append(1)
                    else:
                        P.append(0)
                    Q.append(qc1_state_runs[inv_keys])
                
                Avg_inverted_error_25 = HellingerDistance(P,Q)
                
                P = []
                Q = []
                for inv_keys in qc2_state_runs.keys():
                    if sum([int(x) for x in inv_keys if x!=' '])==0:
                        P.append(1)
                    else:
                        P.append(0)
                    Q.append(qc2_state_runs[inv_keys])
                
                Avg_inverted_error_50 = HellingerDistance(P,Q)
                
                P = []
                Q = []
                for inv_keys in qc3_state_runs.keys():
                    if sum([int(x) for x in inv_keys if x!=' '])==0:
                        P.append(1)
                    else:
                        P.append(0)
                    Q.append(qc3_state_runs[inv_keys])
                
                Avg_inverted_error_75 = HellingerDistance(P,Q)
                
            except Exception as e:
                # print(inverted_state_runs)
                raise(e)
            circuit_depth = circ_trans.depth()
            circuit_width = circ_trans.width()
            ##############################################################################

            
            state_features["circuit_depth"] = []
            state_features["circuit_width"] = []
            # state_features["Avg_inverted_error"] = []
            state_features["Avg_inverted_error_25"] = []
            state_features["Avg_inverted_error_50"] = []
            state_features["Avg_inverted_error_75"] = []
            state_features["circuit_depth"][0:state_len] = [circuit_depth]*state_len
            state_features["circuit_width"][0:state_len] = [circuit_width]*state_len
            # state_features["Avg_inverted_error"][0:state_len] = [np.round(Avg_inverted_error*100,2)]*state_len
            
            #state_features["Avg_observed_prob"] = Avg_observed_prob
            state_features["Avg_inverted_error_25"][0:state_len] = [np.round(Avg_inverted_error_25*100,2)]*state_len
            state_features["Avg_inverted_error_50"][0:state_len] = [np.round(Avg_inverted_error_50*100,2)]*state_len
            state_features["Avg_inverted_error_75"][0:state_len] = [np.round(Avg_inverted_error_75*100,2)]*state_len
            state_features["Avg_odds_ratio"] = []
            state_features["state_weight"] = []
            state_features["observed_prob_25"] = []
            state_features["observed_prob_50"] = []
            state_features["observed_prob_75"] = []
            state_features["target"] = []
            
            noise_state_runs = [noisy_exp_vals_2[cir_index],noisy_exp_vals_3[cir_index]]
            
            for state in noisy.keys():
                
                state_features_state = generate_state_features(state,noise_state_runs)
                if state in list(ideal.keys()):
                    state_features_state["target"] = np.round((ideal[state]/100000)*100,2)
                else:
                    state_features_state["target"] = 0.0
                for key,value in state_features_state.items():
                    if key in state_features:
                        state_features[key].append(state_features_state[key])
            dataframe  =  pd.DataFrame(state_features)
            dataframe.to_csv(f"./multi_backend_model_data/Q_learn_MQT_test/testing_{backend_name}_{qubits}_{cir_index}.csv",index=False)
            # dataframe.to_csv(f"./multi_backend_model_data/Q_learn_test/testing_{backend_name}_{qubits}_{cir_index}.csv",index=False)
        del sim_noise

# Generating data (This may run for over ten hours)

In [6]:
generate_training_data()

  0%|          | 0/6 [00:00<?, ?it/s]

C:\Users\30806\AppData\Local\Temp\ipykernel_26632\2767360837.py:24: FutureWarning: The input object of type 'RZGate' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'RZGate', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  circuits = np.array(circuits)
C:\Users\30806\AppData\Local\Temp\ipykernel_26632\2767360837.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  circuits = np.array(circuits)


In [7]:
generate_test_data()

  0%|          | 0/6 [00:00<?, ?it/s]

C:\Users\30806\AppData\Local\Temp\ipykernel_30192\876582695.py:25: FutureWarning: The input object of type 'SXGate' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'SXGate', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  circuits = np.array(circuits)
C:\Users\30806\AppData\Local\Temp\ipykernel_30192\876582695.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  circuits = np.array(circuits)
C:\Users\30806\AppData\Local\Temp\ipykernel_30192\876582695.py:25: FutureWarning: 